# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [1]:
# Note: if using a virtual environment, do not run this cell
%pip install -U semantic-kernel
from semantic_kernel import __version__

__version__

   ---------------------------------------- 0.0/645.9 kB ? eta -:--:--
   ---------------------------------------- 645.9/645.9 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: semantic-kernel
    Found existing installation: semantic-kernel 1.14.0
    Uninstalling semantic-kernel-1.14.0:
      Successfully uninstalled semantic-kernel-1.14.0
Note: you may need to restart the kernel to use updated packages.


'1.15.0'

Initial configuration for the notebook to run properly.

In [2]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("C:\\Users\\helenzeng\\semantic-kernel\\python\\samples\\getting_started")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

### Configuring the Kernel

Let's get started with the necessary configuration to run Semantic Kernel. For Notebooks, we require a `.env` file with the proper settings for the model you use. Create a new file named `.env` and place it in this directory. Copy the contents of the `.env.example` file from this directory and paste it into the `.env` file that you just created.

**NOTE: Please make sure to include `GLOBAL_LLM_SERVICE` set to either OpenAI, AzureOpenAI, or HuggingFace in your .env file. If this setting is not included, the Service will default to AzureOpenAI.**

#### Option 1: using OpenAI

Add your [OpenAI Key](https://openai.com/product/) key to your `.env` file (org Id only if you have multiple orgs):

```
GLOBAL_LLM_SERVICE="OpenAI"
OPENAI_API_KEY="sk-..."
OPENAI_ORG_ID=""
OPENAI_CHAT_MODEL_ID=""
OPENAI_TEXT_MODEL_ID=""
OPENAI_EMBEDDING_MODEL_ID=""
```
The names should match the names used in the `.env` file, as shown above.

#### Option 2: using Azure OpenAI

Add your [Azure Open AI Service key](https://learn.microsoft.com/azure/cognitive-services/openai/quickstart?pivots=programming-language-studio) settings to the `.env` file in the same folder:

```
GLOBAL_LLM_SERVICE="AzureOpenAI"
AZURE_OPENAI_API_KEY="..."
AZURE_OPENAI_ENDPOINT="https://..."
AZURE_OPENAI_CHAT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_TEXT_DEPLOYMENT_NAME="..."
AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME="..."
AZURE_OPENAI_API_VERSION="..."
```
The names should match the names used in the `.env` file, as shown above.

For more advanced configuration, please follow the steps outlined in the [setup guide](./CONFIGURING_THE_KERNEL.md).

We will load our settings and get the LLM service to use for the notebook.

In [3]:
from services import Service

from samples.service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


In [4]:
from semantic_kernel import Kernel

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    kernel.add_service(
        OpenAIChatCompletion(
            service_id=service_id,
        ),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
        ),
    )

Let's define a prompt outlining a dialogue chat bot.


In [5]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [6]:
from semantic_kernel.connectors.ai.open_ai import AzureChatPromptExecutionSettings, OpenAIChatPromptExecutionSettings
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-4o",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = AzureChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-4o",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [7]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [8]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [9]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Hi there! I'd be happy to help you with that. Could you let me know what genres or types of books you enjoy?


Update the history with the output


In [10]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [11]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [12]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?
ChatBot: Absolutely! Here are a few books that delve into the history and philosophy of Greece:

1. **"The Histories" by Herodotus** - Often considered the "Father of History," Herodotus provides a comprehensive look at the ancient world, including detailed accounts of Greek history and its interactions with other civilizations.

2. **"The Peloponnesian War" by Thucydides** - This classic work offers an in-depth analysis of the war between Athens and Sparta, providing insights into Greek politics, warfare, and society.

3. **"The Republic" by Plato** - A foundational text in Western philosophy, Plato's dialogue explores justice, the ideal state, and the philosopher-king, offering a profound glimpse into Greek philosophical thought.

4. **"The Nicomachean Ethics" by Aristotle** - In this work, Aristotle examines the nature of happiness and virtue, providing a key text in both Greek and Wes

In [13]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?
ChatBot: Sure! Which book are you referring to? I can provide more details about any of the ones I listed.


In [14]:
await chat("if I read that book, what exactly will I learn about Greek history?")

User: if I read that book, what exactly will I learn about Greek history?
ChatBot: Let's break down what you can learn from each of the books I mentioned:

1. **"The Histories" by Herodotus**:
   - **Content**: This book covers the Greco-Persian Wars and provides detailed accounts of various peoples and places in the ancient world.
   - **Key Learnings**: Insights into the causes and events of the Persian Wars, cultural and social practices of different civilizations, and early historical methodology.

2. **"The Peloponnesian War" by Thucydides**:
   - **Content**: Thucydides' work is a detailed historical account of the 27-year conflict between Athens and Sparta.
   - **Key Learnings**: Political and military strategies, the impact of war on Greek society, and the nature of power and human behavior.

3. **"The Republic" by Plato**:
   - **Content**: A philosophical dialogue on justice, the ideal state, and the role of the philosopher.
   - **Key Learnings**: Concepts of justice, the s

In [15]:
await chat("could you list some more books I could read about this topic?")

User: could you list some more books I could read about this topic?
ChatBot: Certainly! Here are some more books that explore various aspects of Greek history and philosophy:

1. **"The Iliad" and "The Odyssey" by Homer** - These epic poems are foundational texts in Greek literature, offering insights into Greek mythology, heroism, and the values of ancient Greek society.

2. **"The Rise of Athens: The Story of the World's Greatest Civilization" by Anthony Everitt** - This book provides a comprehensive history of Athens, from its early days to its height as a cultural and political powerhouse.

3. **"Greek Philosophy: Thales to Aristotle" by Reginald E. Allen** - This anthology includes key works from major Greek philosophers, providing a broad overview of Greek philosophical thought.

4. **"The Greeks: A Global History" by Roderick Beaton** - This book offers a sweeping narrative of Greek history, examining its influence on global history and culture.

5. **"The Birth of Classical Eur

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [16]:
print(chat_history)

<chat_history><message role="system"><text>You are a helpful chatbot who is good about giving book recommendations.</text></message><message role="assistant"><text>Hi there! I'd be happy to help you with that. Could you let me know what genres or types of books you enjoy?</text></message><message role="user"><text>I love history and philosophy, I'd like to learn something new about Greece, any suggestion?</text></message><message role="assistant"><text>Absolutely! Here are a few books that delve into the history and philosophy of Greece:

1. **"The Histories" by Herodotus** - Often considered the "Father of History," Herodotus provides a comprehensive look at the ancient world, including detailed accounts of Greek history and its interactions with other civilizations.

2. **"The Peloponnesian War" by Thucydides** - This classic work offers an in-depth analysis of the war between Athens and Sparta, providing insights into Greek politics, warfare, and society.

3. **"The Republic" by Pla